# Le partitionnement des données

Le partitionnement (*clustering*) est une technique courante des statistiques multivariées pour effectuer des tâches de regroupement entre variables afin de révéler une structure sous-jacente. Il s’agit d’une méthode exploratoire qui aide à la classification des données en regroupant les individus dans des ensembles cohérents où la variance intra-groupes est minimisée quand la variance inter-groupes est, elle, maximisée.

Importons les modules qui seront nécessaires :

In [ ]:
import random
import numpy as np
import pandas as pd

## Une matrice de dissimilarité

### Avec des variables catégorielles

Prenons les réponses de cinq étudiant·es à un test comportant dix questions :

In [ ]:
n_students = 5
n_questions = 10

data = [
    [random.choice(['A', 'B', 'C', 'D']) for _ in range(n_questions)]
    for _ in range(n_students)
]

df = pd.DataFrame(data, index=[f"Student {i+1}" for i in range(n_students)], columns=[f'Q{i+1}' for i in range(0, n_questions)])

display(df)

Et faisons la comparaison deux à deux pour comptabiliser le nombre de fois où leurs réponses divergent. Enfin, normalisons en divisant le résultat par le nombre de questions afin d’obtenir un score entre 0 et 1 où 0 correspond à des étudiant·es aux réponses similaires et 1 des étudiant·es qui n’auront jamais répondu pareil :

In [ ]:
data_array = df.values

# calculate differences between rows
row_diffs = (data_array[:, None] != data_array).sum(axis=2)

# normalize
dissimilarity_matrix = row_diffs / n_questions

# matrix to a DataFrame
dissimilarity_df = pd.DataFrame(dissimilarity_matrix, index=df.index, columns=df.index)

display(dissimilarity_df)

Il est à présent facile d’identifier les paires d’étudiant·es dont les réponses se ressemblent le plus :

In [ ]:
# minimum score > 0 to exclude pairs consisting of the same student
min_score = dissimilarity_df[dissimilarity_df > 0].min().min()

# all the pairs concerned
clusters = dissimilarity_df[dissimilarity_df == min_score].stack().index.tolist()

display(clusters)

### Avec des variables numériques

Dans l’exemple précédent, les variables enregistraient des données catégorielles. Si maintenant nous prenons l’exemple d’une dizaine de textes avec des scores sur 20 dans cinq catégories :

In [ ]:
# 10 texts with a score on 5 categories
df = pd.DataFrame(
    data=np.random.randint(0, 21, size=(10, 5)),
    index=[i + 1 for i in range(0, 10)],
    columns=["Sciences", "Politique", "Littérature", "Journalisme", "Philosophie"]
)

display(df)

Il n’est plus question ici de repérer les catégories où les textes ont obtenu des scores différents, aussi la première étape consiste à calculer une matrice de corrélation :

In [ ]:
correlation_matrix = df.corr()

display(correlation_matrix)

Cette matrice ressort des coefficients variant de -1 à 1 pour exprimer la corrélation entre chaque paire de variables. Pour la transformer en une matrice de dissimilarité, il suffit de calculer l’inverse de la corrélation :

In [ ]:
dissimilarity_matrix = 1 / correlation_matrix

display(dissimilarity_matrix)

Une formule alternative consiste à calculer plutôt l’opposé de la corrélation :

In [ ]:
dissimilarity_matrix = 1 - correlation_matrix

display(dissimilarity_matrix)

Puis à normaliser afin d’obtenir des scores dans l’intervalle $[0,1]$ :

In [ ]:
display(dissimilarity_matrix / 2)